In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset = pd.read_csv('dataset_filepath')
dataset.drop(['id'], axis=1, inplace=True)

In [ ]:
cat_dict = {'rbc':{'normal':0, 'abnormal':1}, 'pc':{'normal':0, 'abnormal':1}, 'pcc':{'notpresent':0, 'present':1}, \
            'ba':{'notpresent':0, 'present':1}, 'htn':{'no':0, 'yes':1}, 'dm':{'no':0, 'yes':1}, 'cad':{'no':0, 'yes':1}, \
            'appet':{'good':0, 'poor':1}, 'pe':{'no':0, 'yes':1}, 'ane':{'no':0, 'yes':1}, \
            'classification':{'notckd':0, 'ckd':1}}

In [ ]:
for col in dataset.columns:
    if col in cat_dict.keys():
        for i in range(len(dataset)):
            if str(dataset[col][i]).strip() in cat_dict[col].keys():
                dataset[col][i] = cat_dict[col][str(dataset[col][i]).strip()]

In [ ]:
dataset.replace({'\t?':np.nan}, inplace=True)

In [ ]:
parameters = pd.DataFrame(dataset.iloc[:, :-1].values)
result = pd.DataFrame(dataset.iloc[:, 24].values)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_parameters = StandardScaler()
t1 = sc_parameters.fit_transform(parameters.iloc[:, :5])
t2 = parameters.iloc[:, 5:9]
t3 = sc_parameters.fit_transform(parameters.iloc[:, 9:18])
t4 = parameters.iloc[:, 18:24]
parameters = np.concatenate([t1, t2, t3, t4], axis=1)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)
parameters = imputer.fit_transform(parameters)

In [ ]:
for x in range(0, 400):
    for y in range(5, 9):
        if parameters[x][y] <= 0.5:
            parameters[x][y] = 0
        elif parameters[x][y] > 0.5:
            parameters[x][y] = 1

for x in range(0, 400):
    for y in range(18, 24):
        if parameters[x][y] <= 0.5:
            parameters[x][y] = 0
        elif parameters[x][y] > 0.5:
            parameters[x][y] = 1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(parameters, result, test_size = 0.25)

In [ ]:
from sklearn.svm import SVC
classifier = SVC()

from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

In [ ]:
classifier = SVC(kernel='linear', C=0.25)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))